In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import re
import distance as dist
import difflib
import jellyfish

In [2]:
df = pd.read_csv('lyrics.csv')

In [3]:
#svi zanrovi
df['genre'].value_counts()

Rock             131377
Pop               49444
Hip-Hop           33965
Not Available     29814
Metal             28408
Other             23683
Country           17286
Jazz              17147
Electronic        16205
R&B                5935
Indie              5732
Folk               3241
Name: genre, dtype: int64

In [4]:
#izdvoji hip hop zanr
rap = df[df['genre']=='Hip-Hop']

In [5]:
rap = rap.dropna()

In [6]:
rap.describe()

,index,year
count,24850.000000,24850.000000
mean,180519.432757,2008.477626
std,106556.277891,25.914608
min,249.000000,67.000000
25%,93865.250000,2006.000000
50%,170124.000000,2008.000000
75%,274391.750000,2012.000000
max,362156.000000,2016.000000


In [7]:
new_rap = rap[rap['year']>2010]

In [8]:
old_rap = rap[rap['year']<2005]

In [9]:
old_rap = old_rap[(old_rap['artist']=='eminem') | (old_rap['artist']=='50-cent') | (old_rap['artist']=='2pac')
                 | (old_rap['artist']=='game') | (old_rap['artist']=='eazy-e')| (old_rap['artist']=='busta-rhymes')
                 | (old_rap['artist']=='beastie-boys') | (old_rap['artist']=='big-tymers') | (old_rap['artist']=='cypress-hill')
                  | (old_rap['artist']=='diplomats')]

In [10]:
new_rap = new_rap[(new_rap['artist']=='drake') | (new_rap['artist']=='chief-keef') | (new_rap['artist']=='french-montana')
                 | (new_rap['artist']=='future') | (new_rap['artist']=='big-sean')| (new_rap['artist']=='asap-rocky')
                 | (new_rap['artist']=='fetty-wap') | (new_rap['artist']=='2-chainz') | (new_rap['artist']=='ab-soul')
                  | (new_rap['artist']=='childish-gambino')]

In [11]:
old_rap_sample = old_rap.sort_values(['artist', 'year']).groupby('artist').head(5)
new_rap_sample = new_rap.sort_values(['artist', 'year'], ascending=False).groupby('artist').head(5)

In [12]:
new_rap_sample['lyrics'] = new_rap_sample['lyrics'].str.lower()
old_rap_sample['lyrics'] = old_rap_sample['lyrics'].str.lower()

In [13]:
new_rap_sample['lyrics'] = np.array([lyrics.strip() for lyrics in new_rap_sample['lyrics'] ])
old_rap_sample['lyrics'] = np.array([lyrics.strip() for lyrics in old_rap_sample['lyrics'] ])

In [14]:
old_rap_sample.head()

,index,song,year,artist,genre,lyrics
126994,126994,letter-to-the-president,1999,2pac,Hip-Hop,uh dear mr. president what's happenin'? i'm wr...
126995,126995,homeboyz,1999,2pac,Hip-Hop,"oh shit, caught that nigga alone\nain't that a..."
126996,126996,u-can-be-touched,1999,2pac,Hip-Hop,"life, what the f*** is life for n****s like us..."
126997,126997,high-speed,1999,2pac,Hip-Hop,[e.d.i amin]\ni speak\nfor all my n****z livin...
126998,126998,secretz-of-war,1999,2pac,Hip-Hop,"you either ride wit' us, or collide wit' us\ni..."


In [15]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marlena/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

#function for lemmatizing
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

[nltk_data] Downloading package wordnet to /Users/marlena/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
old_rap_sample['tokens'] = old_rap_sample[['song','artist', 'lyrics']].apply(lambda row:word_tokenize(row['lyrics']),axis=1)
old_rap_sample['tokens'] = old_rap_sample['tokens'].apply(lambda x: [token for token in x if token not in en_stop])
old_rap_sample['tokens'] = old_rap_sample['tokens'].apply(lambda x: [get_lemma(token) for token in x])
old_rap_sample['tokens'] = old_rap_sample['tokens'].apply(lambda x: [word for word in x if word.isalpha() ])
old_rap_sample.head()

,index,song,year,artist,genre,lyrics,tokens
126994,126994,letter-to-the-president,1999,2pac,Hip-Hop,uh dear mr. president what's happenin'? i'm wr...,"[uh, dear, president, happenin, writin, shit, ..."
126995,126995,homeboyz,1999,2pac,Hip-Hop,"oh shit, caught that nigga alone\nain't that a...","[oh, shit, catch, nigga, alone, ai, bitch, hey..."
126996,126996,u-can-be-touched,1999,2pac,Hip-Hop,"life, what the f*** is life for n****s like us...","[life, life, like, us, waking, another, day, t..."
126997,126997,high-speed,1999,2pac,Hip-Hop,[e.d.i amin]\ni speak\nfor all my n****z livin...,"[amin, speak, livin, rush, slow, notch, baby, ..."
126998,126998,secretz-of-war,1999,2pac,Hip-Hop,"you either ride wit' us, or collide wit' us\ni...","[either, ride, wit, us, collide, wit, us, simp..."


In [18]:
new_rap_sample['tokens'] = new_rap_sample[['song','artist', 'lyrics']].apply(lambda row:word_tokenize(row['lyrics']),axis=1)
new_rap_sample['tokens'] = new_rap_sample['tokens'].apply(lambda x: [token for token in x if token not in en_stop])
new_rap_sample['tokens'] = new_rap_sample['tokens'].apply(lambda x: [get_lemma(token) for token in x])
new_rap_sample['tokens'] = new_rap_sample['tokens'].apply(lambda x: [word for word in x if word.isalpha() ])
new_rap_sample.head()

,index,song,year,artist,genre,lyrics,tokens
170138,170138,radical,2016,future,Hip-Hop,monster!\nfuck up your attitude\nfuck all the ...,"[monster, fuck, attitude, fuck, gratitude, bit..."
170139,170139,seven-rings,2016,future,Hip-Hop,"yeah, i told 'em\ni told 'em if it's a problem...","[yeah, tell, tell, problem, say, problem, tell..."
170140,170140,fly-shit-only,2016,future,Hip-Hop,i take my drugs in doses\ni been waking up to ...,"[take, drug, dose, waking, bitch, hotel, suite..."
170141,170141,low-life,2016,future,Hip-Hop,"high, high, getting high\neverybody getting hi...","[high, high, getting, high, everybody, getting..."
170142,170142,hardly,2016,future,Hip-Hop,"i remember, it was new years\nyou was in the s...","[remember, new, years, studio, chosen, nigga, ..."


In [19]:
new_rap_tokens = new_rap_sample
old_rap_tokens = old_rap_sample

In [20]:
new_rap_sample.head()

,index,song,year,artist,genre,lyrics,tokens
170138,170138,radical,2016,future,Hip-Hop,monster!\nfuck up your attitude\nfuck all the ...,"[monster, fuck, attitude, fuck, gratitude, bit..."
170139,170139,seven-rings,2016,future,Hip-Hop,"yeah, i told 'em\ni told 'em if it's a problem...","[yeah, tell, tell, problem, say, problem, tell..."
170140,170140,fly-shit-only,2016,future,Hip-Hop,i take my drugs in doses\ni been waking up to ...,"[take, drug, dose, waking, bitch, hotel, suite..."
170141,170141,low-life,2016,future,Hip-Hop,"high, high, getting high\neverybody getting hi...","[high, high, getting, high, everybody, getting..."
170142,170142,hardly,2016,future,Hip-Hop,"i remember, it was new years\nyou was in the s...","[remember, new, years, studio, chosen, nigga, ..."


In [21]:
s = old_rap_sample.apply(lambda x: pd.Series(x['tokens']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'tokens'
old_rap_sample = old_rap_sample.drop('tokens', axis=1).join(s)
old_rap_sample['tokens'] = pd.Series(old_rap_sample['tokens'], dtype=object)
old_rap_sample = old_rap_sample[['artist', 'lyrics', 'tokens']]
old_rap_sample.head()

,artist,lyrics,tokens
321,eazy-e,yo man i don't think they heard you\nwhy don't...,yo
321,eazy-e,yo man i don't think they heard you\nwhy don't...,man
321,eazy-e,yo man i don't think they heard you\nwhy don't...,think
321,eazy-e,yo man i don't think they heard you\nwhy don't...,hear
321,eazy-e,yo man i don't think they heard you\nwhy don't...,tell


In [22]:
s1 = new_rap_sample.apply(lambda x: pd.Series(x['tokens']), axis=1).stack().reset_index(level=1, drop=True)
s1.name = 'tokens'
new_rap_sample = new_rap_sample.drop('tokens', axis=1).join(s1)
new_rap_sample['tokens'] = pd.Series(new_rap_sample['tokens'], dtype=object)
new_rap_sample = new_rap_sample[['artist', 'lyrics', 'tokens']]
new_rap_sample.head()

,artist,lyrics,tokens
17936,fetty-wap,"[intro]\nohh, ohhh\nohh, ohhh\n(basskidsontheb...",intro
17936,fetty-wap,"[intro]\nohh, ohhh\nohh, ohhh\n(basskidsontheb...",ohh
17936,fetty-wap,"[intro]\nohh, ohhh\nohh, ohhh\n(basskidsontheb...",ohhh
17936,fetty-wap,"[intro]\nohh, ohhh\nohh, ohhh\n(basskidsontheb...",ohh
17936,fetty-wap,"[intro]\nohh, ohhh\nohh, ohhh\n(basskidsontheb...",ohhh


In [23]:
new_rap_sample = new_rap_sample.groupby(['artist', 'tokens']).count()
new_rap_sample = new_rap_sample.reset_index()
new_rap_sample.columns = ['artist', 'word', 'n']
new_rap_sample = new_rap_sample.sort_values('n', ascending=False)
new_rap_sample.head()

,artist,word,n
4549,future,shit,158
3947,french-montana,keep,80
4512,future,radical,73
3119,drake,like,61
4324,future,fly,61


In [24]:
old_rap_sample = old_rap_sample.groupby(['artist', 'tokens']).count()
old_rap_sample = old_rap_sample.reset_index()
old_rap_sample.columns = ['artist', 'word', 'n']
old_rap_sample = old_rap_sample.sort_values('n', ascending=False)
old_rap_sample.head()

,artist,word,n
3393,busta-rhymes,make,136
3298,busta-rhymes,gon,131
3427,busta-rhymes,na,126
3360,busta-rhymes,la,91
3716,busta-rhymes,yeah,77


In [25]:
#ukupan broj reci u pesmama starijeg hip hopa
total_words = old_rap_sample[['artist','n']].groupby('artist').sum()
total_words = total_words.reset_index()
total_words.columns=['artist', 'total']
total_words.sort_values('total', ascending=False)

,artist,total
0,2pac,2435
4,busta-rhymes,2202
3,big-tymers,1962
5,cypress-hill,1542
2,beastie-boys,1444
6,diplomats,1379
1,50-cent,1354
9,game,1309
7,eazy-e,1242
8,eminem,1232


In [26]:
#ukupan broj jedinstvenih reci u pesmama starijeg hip hopa
total_words_distinct = old_rap_sample[['artist','n']].groupby('artist').count()
total_words_distinct = total_words_distinct.reset_index()
total_words_distinct.columns=['artist', 'total']
total_words_distinct.sort_values('total', ascending=False)

,artist,total
0,2pac,911
5,cypress-hill,833
2,beastie-boys,824
3,big-tymers,696
6,diplomats,693
4,busta-rhymes,689
8,eminem,651
1,50-cent,603
9,game,587
7,eazy-e,579


In [27]:
#ukupan broj reci u pesmama novijeg hip hopa
total_words1 = new_rap_sample[['artist','n']].groupby('artist').sum()
total_words1 = total_words1.reset_index()
total_words1.columns=['artist', 'total']
total_words1.sort_values('total', ascending=False)

,artist,total
1,ab-soul,1834
3,big-sean,1676
9,future,1577
8,french-montana,1528
7,fetty-wap,1323
6,drake,1275
0,2-chainz,1134
2,asap-rocky,930
5,childish-gambino,783
4,chief-keef,735


In [28]:
#ukupan broj jedinstvenih reci u pesmama novijeg hip hopa
total_words_distinct1 = new_rap_sample[['artist','n']].groupby('artist').count()
total_words_distinct1 = total_words_distinct1.reset_index()
total_words_distinct1.columns=['artist', 'total']
total_words_distinct1.sort_values('total', ascending=False)

,artist,total
1,ab-soul,617
3,big-sean,584
0,2-chainz,525
6,drake,507
9,future,497
2,asap-rocky,494
8,french-montana,443
5,childish-gambino,362
7,fetty-wap,337
4,chief-keef,306


In [29]:
old_rap_sample = pd.merge(old_rap_sample, total_words, how='left', on='artist')
new_rap_sample = pd.merge(new_rap_sample, total_words1, how='left', on='artist')

In [30]:
old_rap_sample['rank'] = old_rap_sample.groupby('artist').cumcount()+1
freq_by_rank = old_rap_sample
freq_by_rank['term_frequency'] = freq_by_rank['n']/freq_by_rank['total']
freq_by_rank.head(20)

,artist,word,n,total,rank,term_frequency
0,busta-rhymes,make,136,2202,1,0.061762
1,busta-rhymes,gon,131,2202,2,0.059491
2,busta-rhymes,na,126,2202,3,0.057221
3,busta-rhymes,la,91,2202,4,0.041326
4,busta-rhymes,yeah,77,2202,5,0.034968
5,big-tymers,nigga,64,1962,1,0.032620
6,2pac,get,64,2435,1,0.026283
7,big-tymers,get,64,1962,2,0.032620
8,game,get,61,1309,1,0.046600
9,busta-rhymes,body,56,2202,6,0.025431


In [31]:
new_rap_sample['rank'] = new_rap_sample.groupby('artist').cumcount()+1
freq_by_rank1 = new_rap_sample
freq_by_rank1['term_frequency'] = freq_by_rank1['n']/freq_by_rank1['total']
freq_by_rank1.head(20)

,artist,word,n,total,rank,term_frequency
0,future,shit,158,1577,1,0.100190
1,french-montana,keep,80,1528,1,0.052356
2,future,radical,73,1577,2,0.046290
3,drake,like,61,1275,1,0.047843
4,future,fly,61,1577,3,0.038681
5,big-sean,know,53,1676,1,0.031623
6,ab-soul,love,52,1834,1,0.028353
7,fetty-wap,yeah,48,1323,1,0.036281
8,fetty-wap,boy,48,1323,2,0.036281
9,chief-keef,like,46,735,1,0.062585


In [32]:
for_idf = old_rap_sample.groupby(['word','artist']).count().reset_index().groupby('word').sum().reset_index()
for_idf = for_idf[['word','n']]
for_idf.columns = ['word','foridf']
old_rap_sample = pd.merge(old_rap_sample, for_idf, how='left', on='word')
old_rap_sample.head()

,artist,word,n,total,rank,term_frequency,foridf
0,busta-rhymes,make,136,2202,1,0.061762,10
1,busta-rhymes,gon,131,2202,2,0.059491,8
2,busta-rhymes,na,126,2202,3,0.057221,10
3,busta-rhymes,la,91,2202,4,0.041326,3
4,busta-rhymes,yeah,77,2202,5,0.034968,9


In [33]:
for_idf1 = new_rap_sample.groupby(['word','artist']).count().reset_index().groupby('word').sum().reset_index()
for_idf1 = for_idf1[['word','n']]
for_idf1.columns = ['word','foridf']
new_rap_sample = pd.merge(new_rap_sample, for_idf1, how='left', on='word')
new_rap_sample.head()

,artist,word,n,total,rank,term_frequency,foridf
0,future,shit,158,1577,1,0.100190,10
1,french-montana,keep,80,1528,1,0.052356,10
2,future,radical,73,1577,2,0.046290,1
3,drake,like,61,1275,1,0.047843,10
4,future,fly,61,1577,3,0.038681,7


In [34]:
N = total_words.count()[0] #number of words
old_rap_sample['tf'] = old_rap_sample['n']/old_rap_sample['total'] #calculate tf
old_rap_sample['idf'] = np.log(N*1.0/(1+old_rap_sample['foridf']*1.0)) #calculate idf
old_rap_sample.loc[old_rap_sample['idf'] <0, 'idf'] = 0 #use just non-negative values of idf
old_rap_sample['tf_idf'] = old_rap_sample['tf']*old_rap_sample['idf'] #calculate tf-idf
old_rap_sample.sort_values(by=['tf_idf'], ascending=False).head(10)

,artist,word,n,total,rank,term_frequency,foridf,tf,idf,tf_idf
3,busta-rhymes,la,91,2202,4,0.041326,3,0.041326,0.916291,0.037867
16,eminem,solo,40,1232,1,0.032468,3,0.032468,0.916291,0.029750
55,diplomats,santana,20,1379,2,0.014503,1,0.014503,1.609438,0.023342
79,eazy-e,eazy,17,1242,7,0.013688,1,0.013688,1.609438,0.022029
21,busta-rhymes,wet,35,2202,8,0.015895,2,0.015895,1.203973,0.019137
65,eazy-e,rather,19,1242,5,0.015298,2,0.015298,1.203973,0.018418
39,game,president,26,1309,3,0.019862,3,0.019862,0.916291,0.018200
147,eminem,normale,11,1232,9,0.008929,1,0.008929,1.609438,0.014370
62,busta-rhymes,wildin,19,2202,14,0.008629,1,0.008629,1.609438,0.013887
63,busta-rhymes,rowdy,19,2202,15,0.008629,1,0.008629,1.609438,0.013887


In [35]:
N = total_words1.count()[0] #number of words
new_rap_sample['tf'] = new_rap_sample['n']/new_rap_sample['total'] #calculate tf
new_rap_sample['idf'] = np.log(N*1.0/(1+new_rap_sample['foridf']*1.0)) #calculate idf
new_rap_sample.loc[new_rap_sample['idf'] <0, 'idf'] = 0 #use just non-negative values of idf
new_rap_sample['tf_idf'] = new_rap_sample['tf']*new_rap_sample['idf'] #calculate tf-idf
new_rap_sample.sort_values(by=['tf_idf'], ascending=False).head(10)

,artist,word,n,total,rank,term_frequency,foridf,tf,idf,tf_idf
2,future,radical,73,1577,2,0.046290,1,0.046290,1.609438,0.074502
45,childish-gambino,spinning,24,783,1,0.030651,1,0.030651,1.609438,0.049331
15,french-montana,jaw,40,1528,3,0.026178,1,0.026178,1.609438,0.042132
20,french-montana,bite,36,1528,5,0.023560,1,0.023560,1.609438,0.037919
66,drake,grammy,21,1275,7,0.016471,1,0.016471,1.609438,0.026508
64,chief-keef,rolling,21,735,5,0.028571,3,0.028571,0.916291,0.026180
76,drake,whoa,20,1275,8,0.015686,1,0.015686,1.609438,0.025246
28,future,hardly,33,1577,9,0.020926,2,0.020926,1.203973,0.025194
164,childish-gambino,strong,11,783,6,0.014049,1,0.014049,1.609438,0.022610
135,chief-keef,dolo,13,735,7,0.017687,2,0.017687,1.203973,0.021295


In [36]:
from gensim import corpora
import pickle
import gensim
import pyLDAvis.gensim

#create dictionary and corpus
dictionary = corpora.Dictionary(old_rap_tokens['tokens'])
corpus = [dictionary.doc2bow(text) for text in old_rap_tokens['tokens']]

#save dictionary
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [37]:
NUM_TOPICS = 5 #choose number of topics

#create LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim') #save model

topics = ldamodel.print_topics(num_words=10) #choose number of words per topic

#print topics
for topic in topics:
    print(topic)

(0, '0.014*"get" + 0.014*"fuck" + 0.011*"us" + 0.009*"rather" + 0.009*"president" + 0.009*"life" + 0.009*"niggaz" + 0.007*"tell" + 0.007*"touch" + 0.007*"ai"')
(1, '0.028*"get" + 0.023*"nigga" + 0.018*"la" + 0.013*"shit" + 0.012*"like" + 0.009*"ai" + 0.008*"know" + 0.008*"life" + 0.007*"fuck" + 0.007*"hoe"')
(2, '0.026*"get" + 0.019*"nigga" + 0.014*"like" + 0.008*"shit" + 0.007*"ya" + 0.007*"go" + 0.007*"know" + 0.007*"fuck" + 0.007*"yo" + 0.006*"back"')
(3, '0.010*"get" + 0.008*"steel" + 0.007*"johnny" + 0.006*"ryall" + 0.006*"magnolia" + 0.006*"nigga" + 0.005*"like" + 0.004*"shit" + 0.003*"ai" + 0.003*"go"')
(4, '0.043*"gon" + 0.043*"na" + 0.031*"make" + 0.017*"get" + 0.016*"yeah" + 0.014*"body" + 0.012*"know" + 0.010*"solo" + 0.010*"like" + 0.008*"wet"')


In [39]:
#load dictionary, corpus and model
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

#create visualization of topic modeling
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


In [40]:
from gensim import corpora
import pickle
import gensim
import pyLDAvis.gensim

#create dictionary and corpus
dictionary = corpora.Dictionary(new_rap_tokens['tokens'])
corpus = [dictionary.doc2bow(text) for text in new_rap_tokens['tokens']]

#save dictionary
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [41]:
NUM_TOPICS = 5 #choose number of topics

#create LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim') #save model

topics = ldamodel.print_topics(num_words=10) #choose number of words per topic

#print topics
for topic in topics:
    print(topic)

(0, '0.026*"keep" + 0.022*"get" + 0.018*"ay" + 0.016*"boy" + 0.015*"yeah" + 0.014*"nigga" + 0.014*"bitch" + 0.014*"shit" + 0.013*"hard" + 0.012*"jaw"')
(1, '0.022*"get" + 0.018*"ai" + 0.016*"nigga" + 0.016*"like" + 0.013*"love" + 0.012*"ca" + 0.012*"hear" + 0.012*"know" + 0.011*"never" + 0.009*"see"')
(2, '0.056*"shit" + 0.022*"radical" + 0.019*"fly" + 0.015*"like" + 0.015*"fuck" + 0.013*"keep" + 0.012*"life" + 0.011*"get" + 0.011*"back" + 0.011*"nigga"')
(3, '0.038*"like" + 0.021*"hardly" + 0.016*"get" + 0.013*"know" + 0.013*"fall" + 0.011*"london" + 0.010*"bridge" + 0.008*"shit" + 0.007*"cat" + 0.006*"baby"')
(4, '0.032*"like" + 0.025*"get" + 0.017*"know" + 0.011*"man" + 0.010*"nigga" + 0.010*"go" + 0.010*"baby" + 0.009*"yeah" + 0.009*"fade" + 0.009*"shit"')


In [42]:
#load dictionary, corpus and model
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

#create visualization of topic modeling
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))
